<a href="https://colab.research.google.com/github/gorkemozkaya/nmt-en-tr/blob/master/Turkish_English_NMT_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running predictions with the pre-trained NMT models in Python
This notebook illustrates how one can load the pre-trained models shared in this repo and run them on new Turkish or English sentences for translation. The models are trained using the template provided in TensorFlow 2's official models repository.

First let's install the compatible versions of the dependencies and clone our repository, which includes the customized `models` and `datasets` packages as a dependency.

In [ ]:
%%sh
pip install -q tensorflow==2.8.2 tensorflow-text==2.8.2 tensorflow-addons==0.17.1
[ -d nmt-en-tr ] || git clone -q --recurse-submodules -j8 https://github.com/gorkemozkaya/nmt-en-tr.git
pip3 install -q --user -r /content/nmt-en-tr/models/official/requirements.txt
pip3 install -q -e /content/nmt-en-tr/datasets
[ -e pretrained_v2.zip ] || wget -nc -q https://github.com/gorkemozkaya/nmt-en-tr/releases/download/pretrained_model_v2/pretrained_v2.zip
[ -d pretrained_v2 ] ||  unzip -n -qq pretrained_v2.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 126.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 121.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.8.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.3 which is incompatible.


Update system path

In [ ]:
import sys
sys.path = ['/content/nmt-en-tr/datasets', '/content/nmt-en-tr/models'] + sys.path
sys.path = ['/root/.local/lib/python3.10/site-packages', '/root/.local/bin'] + sys.path

**Load the tokenizer**

In [ ]:
!pip uninstall -y protobuf

Found existing installation: protobuf 4.23.3
Uninstalling protobuf-4.23.3:
  Successfully uninstalled protobuf-4.23.3


In [ ]:
!pip install protobuf==3.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-bigquery 3.10.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-bigquery-connection 1.12.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google

In [ ]:
import tensorflow_text as tftxt
import tensorflow as tf
tokenizer= tftxt.SentencepieceTokenizer(
          model=tf.io.gfile.GFile("pretrained_v2/sentencepiece_en_tr.model", "rb").read(),
          add_eos=True)

Load the keras models

In [ ]:
from official.core import exp_factory, task_factory
from official.nlp.configs import wmt_transformer_experiments as wmt_te

task_config = exp_factory.get_exp_config('transformer_tr_en_blended/base').task
task_config.sentencepiece_model_path = 'pretrained_v2/sentencepiece_en_tr.model'

translation_task = task_factory.get_task(task_config)
model_en_tr = translation_task.build_model()
model_tr_en = translation_task.build_model() # we can use the same task

**Translation wrapper:** Function that does tokenization, translation and detokenization.

In [ ]:
def translate(input_text, model):
  tokenized = tokenizer.tokenize(input_text)
  translated = model({'inputs' : tf.reshape(tokenized, [1, -1])})
  return tokenizer.detokenize(translated['outputs']).numpy()[0].decode('utf-8')

We need to do a dry-run before we can load the weights.

In [ ]:
ignore = translate("test", model_en_tr)
ignore = translate("test", model_tr_en)

In [ ]:
model_en_tr.load_weights("pretrained_v2/en_tr/en_tr")

In [ ]:
model_tr_en.load_weights("pretrained_v2/tr_en/tr_en")

In [ ]:
input = "If Turkey provides a competitive, safe, and predictable business and \
investment environment, it can reach high growth rates and development levels, \
with its alternative tourism opportunities, agriculture, young, educated \
population, and entrepreneurial spirit."

translate(input, model_en_tr)

'Türkiye rekabetçi, güvenli ve öngörülebilir bir iş ve yatırım ortamı sağlarsa, alternatif turizm olanakları, tarım, genç, eğitimli nüfus ve girişimci ruhuyla yüksek büyüme oranları ve kalkınma seviyelerine ulaşabilir.'

In [ ]:
input = "CHP Genel Başkanı Kemal Kılıçdaroğlu, İngiltere'de başbakanlık için \
yarışan Dışişleri Bakanı Liz Truss'ın sığınmacıların Ruanda'ya gönderileceği \
programa Türkiye gibi ülkeleri de ekleyerek genişletmeyi planladığının öne \
sürülmesine tepki gösterdi."

translate(input, model_tr_en)

"Republican People's Party (CHP) Leader Kemal Kilicdaroglu reacted against the idea that Foreign Minister Liz Truss, who competed for prime minister in Britain, planned to expand countries such as Turkey in the programme where asylum seekers will be sent to Rwanda."